<a href="https://colab.research.google.com/github/philadelphia24/Thesis-Job-Recommender-System-/blob/main/Content_Based_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#User profile

**SECOND ALTERNATIVE** -> USER PROFILE (TO BE REVIEWED AND COMPARED)

UPDATE:
Until now, we were using vacancies attributes to generate content-based recommendations by finding vacancies that are similar to each other (based on their descriptions). But sometimes finding similar vacancies might not be enough. 

GOAL:recommendations based on a jobseeker and all the vacancies they have been matched as opposed to a singular vacancy. 

Generate a profile for a jobseeker by aggregating all of the vacancies they have previously matched/rated




In [ ]:
# Import sys module
import sys

#Specify the desired jobseeker'id
#jobseeker_id = 10466
jobseeker_id = input("Enter the jobseeker ID: ")
jobseeker_id = int(jobseeker_id)

#The function first checks whether the jobseeker_id exists in a DataFrame called matches_df
if jobseeker_id not in matches_df['JobseekerID'].unique():
  print(f"Error: Jobseeker {jobseeker_id} doesn't exist or hasn't been matched before")
  sys.exit()

jobseeker_name = matches_df[matches_df['JobseekerID'] == jobseeker_id]['JobseekerName'].values[0]

# Filter the matches_df to include only matches for the desired jobseeker
jobseeker_matches = matches_df[matches_df['JobseekerID'] == jobseeker_id]

# Extract the corresponding VacancyIDs
matched_vacancies = jobseeker_matches['VacancyID'].tolist()

print(matched_vacancies)


Enter the jobseeker ID: 22762
[44760]


In [ ]:
# Find the indices of the matched vacancies in the vacancies_df
#code filters the vacancies_df to include only the row with that VacancyID, extracts the index of that row, and adds it to the matched_indices list. 
#The result is a list of integers representing the indices of the matched vacancies in the vacancies_df.
matched_indices = [vacancies_df[vacancies_df['VacancyID'] == vid].index[0] for vid in matched_vacancies]

# Extract the vectorized data for the matched vacancies
#does this by indexing the vectorized_data array with the matched_indices list. 
matched_vectors = vectorized_data[matched_indices]

# Compute the average values in the vectorized data for each feature.
user_profile = np.asarray(np.mean(matched_vectors, axis=0))
#The array has the same number of elements as the vectorized data for the job vacancies, with each element representing the average similarity score for that particular feature across all of the matched vacancies.
#For example, if there are 10 features in the vectorized data for each job vacancy, the user_profile array will also have 10 elements, with each element representing the average similarity score for that feature across all of the job vacancies that the jobseeker was matched to.

In [ ]:
# Create a new matrix that excludes the matched vacancies
unmatched_vectors = np.delete(vectorized_data.toarray(), matched_indices, axis=0)

In [ ]:
# Compute the cosine similarity between the user profile and the remaining vacancies
similarity_scores = cosine_similarity(user_profile.reshape(1, -1), unmatched_vectors)

# Sort the similarity scores from high to low
sorted_scores_indices = np.argsort(similarity_scores)[0][::-1]


In [ ]:
# Print the most similar jobseeker vacancies preferences
print("Top 5 job matches for jobseeker", jobseeker_name, "with ID", jobseeker_id, ":\n")
top_matches = sorted_scores_indices[:5]
for i in top_matches:
    print("VacancyID:", vacancies_df.iloc[i]['VacancyID'], "Similarity Score:", similarity_scores[0][i])
    print(vacancies_df.iloc[i]['VacancyDescription'], "\n")

Top 5 job matches for jobseeker Claude Hendrickx De Roeck with ID 22762 :

VacancyID: 48245 Similarity Score: 0.9860015083223546
We are looking for an additional Cleaner to strengthen our team in the Cleaning industry with knowledge of Spanish and 5-10 years of experience, preferably with a High School degree. 

VacancyID: 33680 Similarity Score: 0.9860015083223546
We are looking for an additional Finance Manager to strengthen our team in the Construction industry with knowledge of English and 10+ years of experience, preferably with a Bachelor degree. 

VacancyID: 30074 Similarity Score: 0.9039055954081703
We are looking for an additional Data Analyst to strengthen our team in the Hotels and restaurants industry with knowledge of French and 5-10 years of experience, preferably with a Master degree. 

VacancyID: 41627 Similarity Score: 0.6943793467405706
We are looking for an additional Data Analyst to strengthen our team in the Transport industry with knowledge of German and 0-1 years


NEXT:Match even though jobseeker hasent had a match before (hence, do it by itscharacteristics profile)

In [ ]:
def recommend_jobs(jobseeker_id, top_k=5):

     # Check if jobseeker_id is present in matches_df
    if jobseeker_id not in matches_df['JobseekerID'].values:
        print(f"Jobseeker {jobseeker_id} has no past matches.")
        return None

    # Get the job seeker's past matches
    jobseeker_matches = matches_df[matches_df['JobseekerID'] == jobseeker_id]['VacancyID'].values

    if len(jobseeker_matches) == 0:
        print(f"Jobseeker {jobseeker_id} has no past matches.")
        return None

    # Exclude the matched job vacancies from the recommendations
    remaining_vacancies_df = vacancies_df[~vacancies_df['VacancyID'].isin(jobseeker_matches)]

    # Vectorize the remaining job vacancies
    vectorized_remaining_vacancies = vectorizer.transform(remaining_vacancies_df['VacancyDescription'])

    # Compute cosine similarity between past matches and remaining job vacancies
    similarity_scores = cosine_similarity(vectorized_remaining_vacancies, vectorized_data[jobseeker_matches, :]).mean(axis=1)

    # Sort scores in descending order and select top_k vacancies
    top_vacancies_indices = np.argsort(similarity_scores)[::-1][:top_k]
    top_vacancies = remaining_vacancies_df.iloc[top_vacancies_indices]

    return top_vacancies

In [ ]:
recommendations = recommend_jobs(jobseeker_id=22762, top_k=5)
print(recommendations)

IndexError: ignored

# Iteration 1 

**MAIN MODEL**

* change "content" to Sara's description with stop words for consistency and more realistic setting. Hence, add new "descriptions file", but with all columns of vacancies

* Add preprocessing of experience, so that it's in the TF IDF matrix

* (more of an alternative from Iteration 3)

Source: https://www.datacamp.com/tutorial/recommender-systems-python


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
from google.colab import files
uploaded = files.upload()

import io
jobseekers_df= pd.read_csv(io.BytesIO(uploaded['jobseekers.csv']))
vacancies_df= pd.read_csv(io.BytesIO(uploaded['vacancies_description.csv']))
matches_df= pd.read_csv(io.BytesIO(uploaded['matches.csv']))

Saving jobseekers.csv to jobseekers (2).csv
Saving matches.csv to matches (2).csv
Saving vacancies_description.csv to vacancies_description (2).csv


In [ ]:
display(vacancies_df)

,VacancyID,Job Title,Language,Education Level,Industry,Experience,VacancyDescription
0,49442,Cashier,French,High School,Wholesale and retail,5-10,We are looking for an additional Cashier to st...
1,31482,Data Analyst,Dutch,Bachelor,Administration,0-1,We are looking for an additional Data Analyst ...
2,39069,Janitor,French,High School,Cleaning,10+,We are looking for an additional Janitor to st...
3,31129,Finance Manager,Dutch,Master,Cleaning,5-10,We are looking for an additional Finance Manag...
4,44258,Data Analyst,German,Master,Wholesale and retail,0-1,We are looking for an additional Data Analyst ...
...,...,...,...,...,...,...,...
495,48744,Data Analyst,Spanish,Bachelor,Financial intermediation,5-10,We are looking for an additional Data Analyst ...
496,49498,Finance Manager,German,Bachelor,Manufacturing,5-10,We are looking for an additional Finance Manag...
497,30794,Professor,German,PhD,Research,0-1,We are looking for an additional Professor to ...
498,34852,Janitor,English,High School,Cleaning,5-10,We are looking for an additional Janitor to st...


## Preprocessing

In [ ]:
!pip install num2words
from num2words import num2words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer

# Define a function to preprocess the text data
def preprocess_text(text):
    # Convert "High School" to "high_school"
    text = re.sub(r'\bHigh School\b', 'high_school', text) # #added lowercase so that school is not extracted twice, once as upper, once lower case
    # Convert "High school" to "high_school"
    text = re.sub(r'\bHigh\s+school\b', 'high_school', text)
    # Replace integer ranges like "5-10" with "five_to_ten"
    text = re.sub(r'\b(\d+)-(\d+)\b', lambda match: f"{num2words(int(match.group(1)))}_to_{num2words(int(match.group(2)))}", text)
    # Replace "10+" with "ten_plus"
    text = re.sub(r'10\+', 'ten_plus', text)
    return text

In [ ]:
# Merge the matches_df with the vacancies_df and jobseekers_df
matches_vacancies_df = pd.merge(matches_df, vacancies_df, left_on='VacancyID', right_on='VacancyID')
matches_jobseekers_df = pd.merge(matches_df, jobseekers_df, left_on='JobseekerID', right_on='JobseekerID')

In [ ]:
print(matches_jobseekers_df)

       JobseekerID  VacancyID  Rating                  Name Language  \
0            20473      30356       4  Joren Van Campenhout    Dutch   
1            22762      43396       1          Romy Derycke    Dutch   
2            10466      34808       4       Gabriel Lammens   German   
3            10466      36153       2       Gabriel Lammens   German   
4            10466      38808       3       Gabriel Lammens   German   
...            ...        ...     ...                   ...      ...   
45312        21446      40802       3      Sylvia Blommaert   French   
45313        21446      40283       5      Sylvia Blommaert   French   
45314        21446      42948       2      Sylvia Blommaert   French   
45315        21446      27350       2      Sylvia Blommaert   French   
45316        21446      38792       4      Sylvia Blommaert   French   

      Education Level                Industry Experience  
0                 PhD              Technology        2-4  
1                

In [ ]:
print(matches_vacancies_df)

       JobseekerID  VacancyID  Rating        Job Title Language  \
0            20473      30356       4  Finance Manager    Dutch   
1            24472      30356       4  Finance Manager    Dutch   
2            16658      30356       4  Finance Manager    Dutch   
3            10182      30356       5  Finance Manager    Dutch   
4            13789      30356       4  Finance Manager    Dutch   
...            ...        ...     ...              ...      ...   
45312        10629      31068       4  Finance Manager  English   
45313        10007      31068       4  Finance Manager  English   
45314        12063      31068       1  Finance Manager  English   
45315        13127      31068       1  Finance Manager  English   
45316        19192      31068       5  Finance Manager  English   

      Education Level                  Industry Experience  \
0              Master                Technology        10+   
1              Master                Technology        10+   
2        

In [ ]:
vacancies_df["VacancyDescription"].isnull

<bound method Series.isnull of 0      We are looking for an additional Cashier to st...
1      We are looking for an additional Data Analyst ...
2      We are looking for an additional Janitor to st...
3      We are looking for an additional Finance Manag...
4      We are looking for an additional Data Analyst ...
                             ...                        
495    We are looking for an additional Data Analyst ...
496    We are looking for an additional Finance Manag...
497    We are looking for an additional Professor to ...
498    We are looking for an additional Janitor to st...
499    We are looking for an additional Administrativ...
Name: VacancyDescription, Length: 500, dtype: object>

##TF-IDF

In [ ]:
# Create a TfidfVectorizer object with your preferred settings
tfidf = TfidfVectorizer(max_df=0.7, min_df=2, stop_words='english', preprocessor=preprocess_text)

# Use the vectorizer to transform your text data
vacancies_tfidf = tfidf.fit_transform(vacancies_df['VacancyDescription'])

In [ ]:
for text in vacancies_df['VacancyDescription']:
    preprocessed_text = preprocess_text(text)
    print(preprocessed_text[:150])  # Print the first 150 characters of the preprocessed text


We are looking for an additional Cashier to strengthen our team in the Wholesale and retail industry with knowledge of French and five_to_ten years of
We are looking for an additional Data Analyst to strengthen our team in the Administration industry with knowledge of Dutch and zero_to_one years of e
We are looking for an additional Janitor to strengthen our team in the Cleaning industry with knowledge of French and ten_plus years of experience, pr
We are looking for an additional Finance Manager to strengthen our team in the Cleaning industry with knowledge of Dutch and five_to_ten years of expe
We are looking for an additional Data Analyst to strengthen our team in the Wholesale and retail industry with knowledge of German and zero_to_one yea
We are looking for an additional Janitor to strengthen our team in the Cleaning industry with knowledge of German and two_to_four years of experience,
We are looking for an additional Cashier to strengthen our team in the Wholesale and retail in

In [ ]:
# Extract the feature names from the fitted vectorizer object
feature_names = list(tfidf.vocabulary_.keys())

# Print the feature names that have been extracted from the text data
print(feature_names)

['Cashier', 'Wholesale', 'retail', 'French', 'five_to_ten', 'high_school', 'Data', 'Analyst', 'Administration', 'Dutch', 'zero_to_one', 'Bachelor', 'Janitor', 'Cleaning', 'ten_plus', 'Finance', 'Manager', 'Master', 'German', 'two_to_four', 'Manufacturing', 'Spanish', 'Research', 'Logistics', 'Cleaner', 'Administrative', 'Clerk', 'Truck', 'Driver', 'Transport', 'Professor', 'PhD', 'Education', 'Pharmaceutical', 'English', 'Agriculture', 'Technology', 'Hotels', 'restaurants', 'Construction', 'Financial', 'intermediation', 'Health', 'social', 'services', 'Energy']


In [ ]:
vacancies_tfidf.shape

(500, 46)

*  For the 500 job vacancies there are 48 unique terms across all job vacancies. 

*  These dimensions are useful to understand the size of the tf-idf matrices that are being used to compute the cosine similarity between the job vacancies and job seekers. The larger the number of documents and terms, the more computationally expensive it can be to compute the cosine similarity.

=> 8 (job titles) + 5 (languages) + 4(education levels) + 16 (industries) + 4 (years of experience) =37

46, because 
1. Wholesale, retail +1
2. Data, Analyst +1
3. Finance, Manager +1
4. Administrative, Clerk +1
5. Truck, Driver +1
6. Hotels, restaurants +1
7. Financial, intermediation +1
8. Health, social, services +2



Excluding jobseekers who have not rated any vacancy

Note: It is normal that we cannot provide recommendations without a single rating.

In [ ]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(vacancies_tfidf, vacancies_tfidf)
# Since you have used the TF-IDF vectorizer, calculating the dot product between each vector will directly give you the cosine similarity score. 
# Therefore, you will use sklearn's linear_kernel() instead of cosine_similarities() since it is faster.
# Each movie will be a 1x500 column vector where each column will be a similarity score with each vacancy.

In [ ]:
cosine_sim.shape

(500, 500)

In [ ]:
cosine_sim[1]

# cosine_sim[1] gives you an array containing the cosine similarity scores between the first vacancy and all other vacancies in the vacancies_df. 
# The length of the array will be the same as the number of rows in vacancies_df.

array([0.        , 1.        , 0.        , 0.16138958, 0.47102728,
       0.        , 0.        , 0.45574472, 0.12491067, 0.        ,
       0.        , 0.30244206, 0.16019249, 0.57764401, 0.        ,
       0.61510243, 0.        , 0.        , 0.        , 0.        ,
       0.85784152, 0.17114662, 0.        , 0.        , 0.14446734,
       0.        , 0.59704211, 0.33163754, 0.        , 0.15317963,
       0.        , 0.17352448, 0.        , 0.15317963, 0.46861712,
       0.14841769, 0.        , 0.        , 0.        , 0.        ,
       0.14825406, 0.32390423, 0.        , 0.        , 0.        ,
       0.59690621, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.35664681, 0.        , 0.15259666, 0.30505161,
       0.        , 0.46573486, 0.        , 0.14463733, 0.        ,
       0.46669897, 0.        , 0.12281211, 0.        , 0.34152662,
       0.        , 0.16174791, 0.        , 0.18634159, 0.17352448,
       0.        , 0.        , 0.28032171, 0.        , 0.46225

We will define a function that takes in a VacancyID as an input and outputs a list of the 10 most similar vacancies.

In [ ]:
#Construct a reverse map of indices and VacancyIDs
indices = pd.Series(vacancies_df.index, index=vacancies_df['VacancyID']).drop_duplicates()

In [ ]:
indices[:10]

VacancyID
49442    0
31482    1
39069    2
31129    3
44258    4
29771    5
34552    6
35527    7
44603    8
46906    9
dtype: int64

I want to...

* Give top 5 recommendation for a given Jobseeker (where recommendations exclude previously matched vacancies)

1.   I retrieve all vacancies rated by my given jobseeker and take the highest rated item.
2.   Then I recommend the top 5 similar vacancies (given that the jobseeker has not seen them yet)
3. Indicate if a jobseeker has not given a rating before.


###Recommending vacancies

**With printing the similarity score (and still minimum similarity)** (NOW WORKING)

In [ ]:
jobseeker_id = 10978

# Retrieve the VacancyID of the job vacancies matched with the given JobseekerID
matches_for_jobseeker = matches_df[matches_df['JobseekerID'] == jobseeker_id]

# Sort the matches by rating in descending order
sorted_matches = matches_for_jobseeker.sort_values(by='Rating', ascending=False)

# Select the VacancyID of the top match for the jobseeker
top_match_vacancy_id = sorted_matches.iloc[0]['VacancyID']
print(f"The top match for jobseeker {jobseeker_id} is Vacancy {top_match_vacancy_id}")

# Find the row with Jobseeker and print the JobTitle and Industry columns
print(f'The characteristics of Jobseeker {jobseeker_id} are:')
row_j = jobseekers_df.loc[jobseekers_df['JobseekerID'] == jobseeker_id, ['JobseekerID', 'Industry', 'Language', 'Experience', 'Education Level']]
display(row_j)

# Find the row with the highest rated VacancyID and print the JobTitle and Industry columns
print(f'The characteristics of the highest rated Vacancy {top_match_vacancy_id} are:')
row_v = vacancies_df.loc[vacancies_df['VacancyID'] == top_match_vacancy_id, ['VacancyID','Job Title', 'Industry', 'Language', 'Experience', 'Education Level']]
display(row_v)


# Check if there are any matches for the jobseeker
if not sorted_matches.empty:
    # Get the VacancyIDs that the jobseeker has already rated
    rated_vacancy_ids = sorted_matches['VacancyID'].values

    # Define the modified get_recommendations function
    def get_recommendations(VacancyID, cosine_sim=cosine_sim, min_similarity=0.4): #ADDED: min_similarity=...
        # Get the index of the movie that matches the VacancyID
        idx = indices[VacancyID]

        # Get the pairwise similarity scores of all movies with that vacancy
        sim_scores = list(enumerate(cosine_sim[idx]))
        # ADDED: Filter the similarity scores by the minimum threshold
        sim_scores = [score for score in sim_scores if score[1] >= min_similarity]
       
        # Sort the vacancies based on the similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Get the scores and indices of the 10 most similar vacancies
        sim_scores = sim_scores[1:11]
        vacancy_indices = [i[0] for i in sim_scores] #NEW
        cosine_sim_scores = [i[1] for i in sim_scores] #NEW

        # Get the vacancy indices
        #vacancy_indices = [i[0] for i in sim_scores] REMOVED

        # Remove the VacancyIDs that the jobseeker has already rated
        vacancy_indices = [i for i in vacancy_indices if vacancies_df.iloc[i]['VacancyID'] not in rated_vacancy_ids]

        # Get the details of the most similar vacancies
        #vacancies = vacancies_df[['VacancyID', 'Job Title','Industry','Language','Experience','Education Level']].iloc[vacancy_indices]
       
        # Get the details of the most similar vacancies, including the cosine similarity scores
        vacancies = pd.DataFrame(list(zip(vacancy_indices, cosine_sim_scores)), columns=['index', 'cosine_similarity_score'])
        vacancies = vacancies.merge(vacancies_df[['VacancyID', 'Job Title','Industry','Language','Experience','Education Level']], how='left', left_on='index', right_index=True)
        
        # Return the top 10 most similar vacancies
        #return vacancies

        # Return the top 10 most similar vacancies
        return vacancies[['VacancyID', 'Job Title','Industry','Language','Experience','Education Level', 'cosine_similarity_score']]

    # Get the recommendations for the jobseeker
    recommendations = get_recommendations(top_match_vacancy_id, cosine_sim=cosine_sim, min_similarity=0.4)
  

    print(f"Recommended vacancies based on Vacancy {top_match_vacancy_id} are:")
    display(recommendations)
else:
    print(f"No matches found for jobseeker {jobseeker_id}")

The top match for jobseeker 10978 is Vacancy 44088
The characteristics of Jobseeker 10978 are:


,JobseekerID,Industry,Language,Experience,Education Level
3,10978,Construction,French,10+,High School


The characteristics of the highest rated Vacancy 44088 are:


,VacancyID,Job Title,Industry,Language,Experience,Education Level
115,44088,Finance Manager,Construction,French,10+,Master


Recommended vacancies based on Vacancy 44088 are:


,VacancyID,Job Title,Industry,Language,Experience,Education Level,cosine_similarity_score
0,34121,Finance Manager,Construction,English,10+,Master,0.883693
1,36207,Finance Manager,Construction,English,10+,Master,0.883693
2,30660,Finance Manager,Construction,German,10+,Master,0.880462
3,33680,Finance Manager,Construction,English,10+,Bachelor,0.751207
4,46107,Finance Manager,Construction,German,10+,Bachelor,0.748380
5,43627,Finance Manager,Research,French,10+,Master,0.709840
6,49003,Finance Manager,Wholesale and retail,French,10+,Master,0.703954
7,28211,Finance Manager,Agriculture,French,10+,Master,0.642154
8,31080,Finance Manager,Manufacturing,French,10+,Master,0.625831


###Recommending jobseekers

**Recommending jobseekers to vacancies** (SHOULD WORK NOW!)

*Note: Only the last code box is different*

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
!pip install num2words
from num2words import num2words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import files
uploaded = files.upload()

import io
jobseekers_df= pd.read_csv(io.BytesIO(uploaded['jobseekers.csv']))
vacancies_df= pd.read_csv(io.BytesIO(uploaded['vacancies_description.csv']))
matches_df= pd.read_csv(io.BytesIO(uploaded['matches.csv']))

Saving jobseekers.csv to jobseekers (9).csv
Saving matches.csv to matches (8).csv
Saving vacancies_description.csv to vacancies_description (9).csv


In [ ]:
#print the number of null values and empty strings in the 'Experience' column
print(jobseekers_df['Experience'].isnull().sum())
print((jobseekers_df['Experience'] == '').sum())


0
0


Preprocessing

In [ ]:
# Define a function to preprocess the 'Experience' column
def preprocess_experience(exp_str):
    # Split the experience string into a list of integers or a string '10+'
    # Split the experience string into a list of integers or a string '10+'
    # If the first integer is 0 and the second integer is 1, then the experience falls under the 'Exp_0_1' category, 
    # and the corresponding value in exp_dict is set to 1
    exp_list = [int(s.strip()) if s != '10+' else s.strip() for s in exp_str.split('-')]

    # Create a dictionary mapping each experience category to a binary value
    exp_dict = {'Exp_0_1': 0, 'Exp_2_4': 0, 'Exp_5_10': 0, 'Exp_10_plus': 0}
    if exp_list[0] == 0 and exp_list[1] == 1:
        exp_dict['Exp_0_1'] = 1
    elif exp_list[0] == 2 and exp_list[1] == 4:
        exp_dict['Exp_2_4'] = 1
    elif exp_list[0] == 5 and exp_list[1] == 10:
        exp_dict['Exp_5_10'] = 1
    elif len(exp_list) == 1 and exp_list[0] == '10+':
        exp_dict['Exp_10_plus'] = 1
    
    # Return the dictionary
    return exp_dict

In [ ]:
# Apply the preprocess_experience function to the 'Experience' column of both the jobseekers_df and vacancies_df
jobseekers_df = pd.concat([jobseekers_df.drop('Experience', axis=1), jobseekers_df['Experience'].apply(preprocess_experience).apply(pd.Series)], axis=1)
vacancies_df = pd.concat([vacancies_df.drop('Experience', axis=1), vacancies_df['Experience'].apply(preprocess_experience).apply(pd.Series)], axis=1)

In [ ]:
print(jobseekers_df)

      JobseekerID                    Name Language Education Level  \
0           20473    Joren Van Campenhout    Dutch             PhD   
1           22762            Romy Derycke    Dutch             PhD   
2           10466         Gabriel Lammens   German             PhD   
3           10978          Nicole Bertels   French     High School   
4           15303  Kathleen Vansteenkiste   French     High School   
...           ...                     ...      ...             ...   
8995        18299        Denise Van Herck  Spanish     High school   
8996        17686     Valentina Verhaegen  English     High School   
8997        18212        Yvette De Backer    Dutch             PhD   
8998        23689             Maria Bonte    Dutch     High School   
8999        21446        Sylvia Blommaert   French     High school   

                        Industry  Exp_0_1  Exp_2_4  Exp_5_10  Exp_10_plus  
0                     Technology        0        1         0            0  
1      

In [ ]:
# Convert numeric experience columns to text
jobseekers_df['Exp_0_1'] = jobseekers_df['Exp_0_1'].apply(lambda x: 'zero_one' if x == 1 else 'none')
jobseekers_df['Exp_2_4'] = jobseekers_df['Exp_2_4'].apply(lambda x: 'two_four' if x == 1 else 'none')
jobseekers_df['Exp_5_10'] = jobseekers_df['Exp_5_10'].apply(lambda x: 'five_ten' if x == 1 else 'none')
jobseekers_df['Exp_10_plus'] = jobseekers_df['Exp_10_plus'].apply(lambda x: 'ten_plus' if x == 1 else 'none')

# Combine columns into 'content', excluding the "none" values
jobseekers_df['content'] = jobseekers_df[['Language', 'Education Level', 'Industry', 'Exp_0_1', 'Exp_2_4', 'Exp_5_10', 'Exp_10_plus']].astype(str).apply(lambda x: ' '.join([val for val in x if val != 'none']), axis=1)

In [ ]:
#NOT BEING IMPLEMENTED, DONT KNOW WHY!
# Define a function to preprocess the text data
def preprocess_text(text):
    # Convert "High School" to "high_school"
    text = re.sub(r'\bHigh School\b', 'high_school', text) # #added lowercase so that school is not extracted twice, once as upper, once lower case
    # Convert "High school" to "high_school"
    text = re.sub(r'\bHigh\s+school\b', 'high_school', text)
    # Replace integer ranges like "5-10" with "five_to_ten"

    # Return the dictionary
    return text

In [ ]:
display(jobseekers_df)

,JobseekerID,Name,Language,Education Level,Industry,Exp_0_1,Exp_2_4,Exp_5_10,Exp_10_plus,content
0,20473,Joren Van Campenhout,Dutch,PhD,Technology,none,two_four,none,none,Dutch PhD Technology two_four
1,22762,Romy Derycke,Dutch,PhD,Hotels and restaurants,zero_one,none,none,none,Dutch PhD Hotels and restaurants zero_one
2,10466,Gabriel Lammens,German,PhD,Administration,none,none,none,ten_plus,German PhD Administration ten_plus
3,10978,Nicole Bertels,French,High School,Construction,none,none,none,ten_plus,French High School Construction ten_plus
4,15303,Kathleen Vansteenkiste,French,High School,Manufacturing,zero_one,none,none,none,French High School Manufacturing zero_one
...,...,...,...,...,...,...,...,...,...,...
8995,18299,Denise Van Herck,Spanish,High school,Cleaning,zero_one,none,none,none,Spanish High school Cleaning zero_one
8996,17686,Valentina Verhaegen,English,High School,Health and social services,none,two_four,none,none,English High School Health and social services...
8997,18212,Yvette De Backer,Dutch,PhD,Logistics,none,none,five_ten,none,Dutch PhD Logistics five_ten
8998,23689,Maria Bonte,Dutch,High School,Research,none,none,none,ten_plus,Dutch High School Research ten_plus


In [ ]:
# Step 2: Define a similarity metric to measure the similarity between jobseekers and vacancies
# Create a TfidfVectorizer object to convert the 'content' column into a tf-idf matrix
tfidf = TfidfVectorizer(max_df=0.7, min_df=2, stop_words='english')

In [ ]:
jobseekers_tfidf = tfidf.fit_transform(jobseekers_df['content'])

In [ ]:
jobseekers_tfidf.shape

(9000, 35)

In [ ]:
# Extract the feature names from the fitted vectorizer object
feature_names = list(tfidf.vocabulary_.keys())

# Print the feature namesthat have been extracted from the text data
print(feature_names)

['dutch', 'phd', 'technology', 'two_four', 'hotels', 'restaurants', 'zero_one', 'german', 'administration', 'ten_plus', 'french', 'high', 'school', 'construction', 'manufacturing', 'research', 'five_ten', 'pharmaceutical', 'spanish', 'bachelor', 'education', 'master', 'health', 'social', 'services', 'english', 'transport', 'logistics', 'agriculture', 'energy', 'cleaning', 'wholesale', 'retail', 'financial', 'intermediation']


=> 5 (languages) + 4(education levels) + 16 (industries) + 4 (years of experience) = 29

1.  Hotels and restaurants, +1
2.  High school, +1
3.  Health, social, services + 2
4.  Wholesale and retail +1
5.  Financial intermediation, +1



In [ ]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(jobseekers_tfidf, jobseekers_tfidf)
# Since you have used the TF-IDF vectorizer, calculating the dot product between each vector will directly give you the cosine similarity score. 
# Therefore, you will use sklearn's linear_kernel() instead of cosine_similarities() since it is faster.
# Each vacancy will be a 1x500 column vector where each column will be a similarity score with each vacancy.

In [ ]:
cosine_sim[1]

# cosine_sim[1] gives you an array containing the cosine similarity scores between the first vacancy and all other vacancies in the vacancies_df. 
# The length of the array will be the same as the number of rows in vacancies_df.

array([0.32204352, 1.        , 0.15078848, ..., 0.32368253, 0.16226562,
       0.        ])

In [ ]:
#Construct a reverse map of indices and VacancyIDs
indices = pd.Series(jobseekers_df.index, index=jobseekers_df['JobseekerID']).drop_duplicates()

In [ ]:
indices[:10]

JobseekerID
20473    0
22762    1
10466    2
10978    3
15303    4
19653    5
10201    6
15237    7
14696    8
15745    9
dtype: int64

In [ ]:
vacancy_id = 30356

# Select the JobseekerID of the top match for the Vacancy
top_match_jobseeker_id = sorted_matches.iloc[0]['JobseekerID']
print(f"The top match for Vacancy {vacancy_id } is Jobseeker {top_match_jobseeker_id}")

# Find the row with Vacancy and print the JobTitle and Industry columns
print(f'The characteristics of Vacancy {vacancy_id} are:')
row_j = vacancies_df.loc[vacancies_df['VacancyID'] == vacancy_id, ['VacancyID', 'Industry', 'Language', 'Education Level']]
display(row_j)

# Retrieve the JobseekerID of the jobseekers matched with the given VacancyID
matches_for_vacancy = matches_df[matches_df['VacancyID'] == vacancy_id]

# Sort the matches by rating in descending order
sorted_matches = matches_for_vacancy.sort_values(by='Rating', ascending=False)

# Find the row where the Jobseeker gave the highest rating for the given VacancyID and print the Language, Industry etc. columns
print(f'The characteristics of the Jobseeker who gave the highest rating are {top_match_jobseeker_id} are:')
row_v = jobseekers_df.loc[jobseekers_df['JobseekerID'] == top_match_jobseeker_id, ['JobseekerID', 'Industry', 'Language', 'Education Level']]
display(row_v)

# Check if there are any matches for the vacancy
if not sorted_matches.empty:
    # Get the JobseekerIDs that have already been matched
    matched_jobseeker_ids = sorted_matches['JobseekerID'].values

    # Define the modified get_recommendations function
    def get_recommendations(JobseekerID, cosine_sim=cosine_sim, min_similarity=0.3): #ADDED: min_similarity=...
        # Get the index of the Jobseeker that matches the JobseekerID
        idx = indices[JobseekerID]

        # Get the pairwise similarity scores of all movies with that vacancy
        sim_scores = list(enumerate(cosine_sim[idx]))

        # ADDED: Filter the similarity scores by the minimum threshold
        sim_scores = [score for score in sim_scores if score[1] >= min_similarity]

        # Sort the vacancies based on the similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        # Get the scores of the 10 most similar vacancies
        sim_scores = sim_scores[1:11]
        jobseeker_indices = [i[0] for i in sim_scores] #NEW
        cosine_sim_scores = [i[1] for i in sim_scores] #NEW

        # Get the jobseeker indices
        #vacancy_indices = [i[0] for i in sim_scores] REMOVED

        # Remove the JobseekerIDs that have already been matched with the VacancyID
        jobseeker_indices = [i for i in jobseeker_indices if jobseekers_df.iloc[i]['JobseekerID'] not in matched_jobseeker_ids]

        # Get the details of the most similar vacancies, including the cosine similarity scores
        jobseekers = pd.DataFrame(list(zip(jobseeker_indices, cosine_sim_scores)), columns=['index', 'cosine_similarity_score'])
        jobseekers = jobseekers.merge(jobseekers_df[['JobseekerID','Industry','Language', 'Education Level']], how='left', left_on='index', right_index=True)
    
        # Return the top 10 most similar vacancies
        return jobseekers[['JobseekerID', 'Industry','Language','Education Level', 'cosine_similarity_score']]

    # Get the recommendations for the vacancy
    recommendations = get_recommendations(top_match_jobseeker_id, cosine_sim=cosine_sim, min_similarity=0.3)
  
    print(f"Recommended jobseekers for Vacancy {vacancy_id} based on Jobseeker {top_match_jobseeker_id} are:")
    display(recommendations)
else:
    print(f"No matches found for vacancy {vacancy_id}")

The top match for Vacancy 30356 is Jobseeker 23493
The characteristics of Vacancy 30356 are:


,VacancyID,Industry,Language,Education Level
296,30356,Technology,Dutch,Master


The characteristics of the Jobseeker who gave the highest rating are 23493 are:


,JobseekerID,Industry,Language,Education Level
7525,23493,Administration,French,PhD


Recommended jobseekers for Vacancy 30356 based on Jobseeker 23493 are:


,JobseekerID,Industry,Language,Education Level,cosine_similarity_score
0,23793,Administration,French,PhD,1.000000
1,18945,Administration,French,PhD,1.000000
2,15199,Administration,French,PhD,1.000000
3,23493,Administration,French,PhD,1.000000
4,18456,Administration,French,PhD,1.000000
5,22671,Administration,French,PhD,0.822534
6,13564,Administration,French,PhD,0.822534
7,15320,Administration,French,PhD,0.822534
8,23201,Administration,French,PhD,0.822534
9,20697,Administration,French,PhD,0.822534


# Iteration 2

* Adding an evaluation
* Not yet complete (Work In Progress), I tried applying the train_df instead of matches_df

Source: https://www.datacamp.com/tutorial/recommender-systems-python


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from num2words import num2words
from sklearn.metrics.pairwise import linear_kernel
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [ ]:
from google.colab import files
uploaded = files.upload()

import io
jobseekers_df= pd.read_csv(io.BytesIO(uploaded['jobseekers.csv']))
vacancies_df= pd.read_csv(io.BytesIO(uploaded['vacancies_description.csv']))
matches_df= pd.read_csv(io.BytesIO(uploaded['matches.csv']))

Saving jobseekers.csv to jobseekers (2).csv
Saving matches.csv to matches (2).csv
Saving vacancies_description.csv to vacancies_description (2).csv


In [ ]:
display(vacancies_df)

,VacancyID,Job Title,Language,Education Level,Industry,Experience,VacancyDescription
0,49442,Cashier,French,High School,Wholesale and retail,5-10,We are looking for an additional Cashier to st...
1,31482,Data Analyst,Dutch,Bachelor,Administration,0-1,We are looking for an additional Data Analyst ...
2,39069,Janitor,French,High School,Cleaning,10+,We are looking for an additional Janitor to st...
3,31129,Finance Manager,Dutch,Master,Cleaning,5-10,We are looking for an additional Finance Manag...
4,44258,Data Analyst,German,Master,Wholesale and retail,0-1,We are looking for an additional Data Analyst ...
...,...,...,...,...,...,...,...
495,48744,Data Analyst,Spanish,Bachelor,Financial intermediation,5-10,We are looking for an additional Data Analyst ...
496,49498,Finance Manager,German,Bachelor,Manufacturing,5-10,We are looking for an additional Finance Manag...
497,30794,Professor,German,PhD,Research,0-1,We are looking for an additional Professor to ...
498,34852,Janitor,English,High School,Cleaning,5-10,We are looking for an additional Janitor to st...


In [ ]:
# Merge the matches_df with the vacancies_df and jobseekers_df
matches_vacancies_df = pd.merge(matches_df, vacancies_df, left_on='VacancyID', right_on='VacancyID')
matches_jobseekers_df = pd.merge(matches_df, jobseekers_df, left_on='JobseekerID', right_on='JobseekerID')

In [ ]:
display(matches_vacancies_df)

,JobseekerID,VacancyID,Rating,Job Title,Language,Education Level,Industry,Experience,VacancyDescription
0,20473,30356,4,Finance Manager,Dutch,Master,Technology,10+,We are looking for an additional Finance Manag...
1,24472,30356,4,Finance Manager,Dutch,Master,Technology,10+,We are looking for an additional Finance Manag...
2,16658,30356,4,Finance Manager,Dutch,Master,Technology,10+,We are looking for an additional Finance Manag...
3,10182,30356,5,Finance Manager,Dutch,Master,Technology,10+,We are looking for an additional Finance Manag...
4,13789,30356,4,Finance Manager,Dutch,Master,Technology,10+,We are looking for an additional Finance Manag...
...,...,...,...,...,...,...,...,...,...
45312,10629,31068,4,Finance Manager,English,Bachelor,Financial intermediation,5-10,We are looking for an additional Finance Manag...
45313,10007,31068,4,Finance Manager,English,Bachelor,Financial intermediation,5-10,We are looking for an additional Finance Manag...
45314,12063,31068,1,Finance Manager,English,Bachelor,Financial intermediation,5-10,We are looking for an additional Finance Manag...
45315,13127,31068,1,Finance Manager,English,Bachelor,Financial intermediation,5-10,We are looking for an additional Finance Manag...


In [ ]:
display(matches_jobseekers_df)

,JobseekerID,VacancyID,Rating,Name,Language,Education Level,Industry,Experience
0,20473,30356,4,Joren Van Campenhout,Dutch,PhD,Technology,2-4
1,22762,43396,1,Romy Derycke,Dutch,PhD,Hotels and restaurants,0-1
2,10466,34808,4,Gabriel Lammens,German,PhD,Administration,10+
3,10466,36153,2,Gabriel Lammens,German,PhD,Administration,10+
4,10466,38808,3,Gabriel Lammens,German,PhD,Administration,10+
...,...,...,...,...,...,...,...,...
45312,21446,40802,3,Sylvia Blommaert,French,High school,Cleaning,5-10
45313,21446,40283,5,Sylvia Blommaert,French,High school,Cleaning,5-10
45314,21446,42948,2,Sylvia Blommaert,French,High school,Cleaning,5-10
45315,21446,27350,2,Sylvia Blommaert,French,High school,Cleaning,5-10


## Preprocessing

In [ ]:
!pip install num2words
from num2words import num2words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Define a function to preprocess the text data
def preprocess_text(text):
    # Convert "High School" to "high_school"
    text = re.sub(r'\bHigh School\b', 'high_school', text) #added so that school is not extracted twice, once as upper, once lower case
    # Convert "High school" to "high_school"
    text = re.sub(r'\bHigh\s+school\b', 'high_school', text)
    # Replace integer ranges like "5-10" with "five_to_ten"
    text = re.sub(r'\b(\d+)-(\d+)\b', lambda match: f"{num2words(int(match.group(1)))}_to_{num2words(int(match.group(2)))}", text)
    # Replace "10+" with "ten_plus"
    text = re.sub(r'10\+', 'ten_plus', text)
    return text

**Train-Test split**

In [ ]:
#1. Split the data into training and testing sets with a 80:20 ratio
#train_df, test_df = train_test_split(matches_vacancies_df, test_size=0.2, random_state=42)

In [ ]:
display(train_df)

,JobseekerID,VacancyID,Rating,Job Title,Language,Education Level,Industry,Experience,VacancyDescription
39361,20795,35371,4,Cashier,English,High School,Wholesale and retail,10+,We are looking for an additional Cashier to st...
9027,17144,37683,1,Administrative Clerk,Spanish,Bachelor,Administration,2-4,We are looking for an additional Administrativ...
12027,15830,27035,4,Truck Driver,Spanish,High School,Transport,0-1,We are looking for an additional Truck Driver ...
19242,15465,30264,4,Data Analyst,Dutch,Master,Education,2-4,We are looking for an additional Data Analyst ...
32685,10854,31508,1,Professor,English,PhD,Research,2-4,We are looking for an additional Professor to ...
...,...,...,...,...,...,...,...,...,...
11284,20841,36071,1,Truck Driver,Spanish,High School,Transport,5-10,We are looking for an additional Truck Driver ...
44732,10207,36942,2,Professor,German,PhD,Research,0-1,We are looking for an additional Professor to ...
38158,14462,25304,2,Truck Driver,French,High School,Transport,2-4,We are looking for an additional Truck Driver ...
860,12729,25023,4,Administrative Clerk,German,High school,Administration,10+,We are looking for an additional Administrativ...


In [ ]:
display(test_df)

,JobseekerID,VacancyID,Rating,Job Title,Language,Education Level,Industry,Experience,VacancyDescription
35452,14283,30629,5,Truck Driver,Dutch,High School,Transport,10+,We are looking for an additional Truck Driver ...
24887,16108,36419,1,Finance Manager,German,Bachelor,Technology,10+,We are looking for an additional Finance Manag...
12296,23714,46816,5,Truck Driver,Spanish,High School,Transport,0-1,We are looking for an additional Truck Driver ...
5034,14594,44465,4,Truck Driver,English,High School,Transport,10+,We are looking for an additional Truck Driver ...
8677,21590,30556,5,Administrative Clerk,Spanish,High school,Administration,10+,We are looking for an additional Administrativ...
...,...,...,...,...,...,...,...,...,...
20083,10344,40080,1,Finance Manager,German,Bachelor,Agriculture,5-10,We are looking for an additional Finance Manag...
7722,24858,34155,4,Data Analyst,French,Master,Pharmaceutical,2-4,We are looking for an additional Data Analyst ...
33554,23888,28143,3,Finance Manager,English,Bachelor,Technology,5-10,We are looking for an additional Finance Manag...
1357,15667,35983,4,Administrative Clerk,German,High school,Administration,5-10,We are looking for an additional Administrativ...


In [ ]:
# 2. Split the test set into actual and predicted dataframes
#actual_df = test_df[['JobseekerID', 'VacancyID', 'Rating']]
#predicted_df = pd.DataFrame(columns=['JobseekerID', 'VacancyID', 'PredictedRating'])

##TF-IDF

In [ ]:
# Create a TfidfVectorizer object with your preferred settings
tfidf = TfidfVectorizer(max_df=0.7, min_df=2, stop_words='english', preprocessor=preprocess_text)

# Use the vectorizer to transform your text data
vacancies_tfidf = tfidf.fit_transform(vacancies_df['VacancyDescription'])

In [ ]:
#SKIP, TAKES LONG!: This was just to check how the vacancy description is read. 
#for text in train_df['VacancyDescription']:
   # preprocessed_text = preprocess_text(text)
   # display(preprocessed_text[:100])  # Print the first 150 characters of the preprocessed text

In [ ]:
# Extract the feature names from the fitted vectorizer object
feature_names = list(tfidf.vocabulary_.keys())

# Print the feature names that have been extracted from the text data
print(feature_names)

['Cashier', 'Wholesale', 'retail', 'French', 'five_to_ten', 'high_school', 'Data', 'Analyst', 'Administration', 'Dutch', 'zero_to_one', 'Bachelor', 'Janitor', 'Cleaning', 'ten_plus', 'Finance', 'Manager', 'Master', 'German', 'two_to_four', 'Manufacturing', 'Spanish', 'Research', 'Logistics', 'Cleaner', 'Administrative', 'Clerk', 'Truck', 'Driver', 'Transport', 'Professor', 'PhD', 'Education', 'Pharmaceutical', 'English', 'Agriculture', 'Technology', 'Hotels', 'restaurants', 'Construction', 'Financial', 'intermediation', 'Health', 'social', 'services', 'Energy']


In [ ]:
vacancies_tfidf.shape

(500, 46)

*  For the 500 job vacancies there are 48 unique terms across all job vacancies. 

*  These dimensions are useful to understand the size of the tf-idf matrices that are being used to compute the cosine similarity between the job vacancies and job seekers. The larger the number of documents and terms, the more computationally expensive it can be to compute the cosine similarity.

=> 8 (job titles) + 5 (languages) + 4(education levels) + 16 (industries) + 4 (years of experience) =37

46, because 
1. Wholesale, retail +1
2. Data, Analyst +1
3. Finance, Manager +1
4. Administrative, Clerk +1
5. Truck, Driver +1
6. Hotels, restaurants +1
7. Financial, intermediation +1
8. Health, social, services +2



In [ ]:
# Compute the cosine similarity matrix
cosine_sim = linear_kernel(vacancies_tfidf, vacancies_tfidf)

# Since you have used the TF-IDF vectorizer, calculating the dot product between each vector will directly give you the cosine similarity score. 
# Therefore, you will use sklearn's linear_kernel() instead of cosine_similarities() since it is faster.
# Each vacancy will be a 1x500 column vector where each column will be a similarity score with each vacancy.

In [ ]:
cosine_sim.shape

(500, 500)

In [ ]:
cosine_sim[1]

# cosine_sim[1] gives you an array containing the cosine similarity scores between the first vacancy and all other vacancies in the vacancies_df. 
# The length of the array will be the same as the number of rows in vacancies_df.

array([0.        , 1.        , 0.        , 0.16138958, 0.47102728,
       0.        , 0.        , 0.45574472, 0.12491067, 0.        ,
       0.        , 0.30244206, 0.16019249, 0.57764401, 0.        ,
       0.61510243, 0.        , 0.        , 0.        , 0.        ,
       0.85784152, 0.17114662, 0.        , 0.        , 0.14446734,
       0.        , 0.59704211, 0.33163754, 0.        , 0.15317963,
       0.        , 0.17352448, 0.        , 0.15317963, 0.46861712,
       0.14841769, 0.        , 0.        , 0.        , 0.        ,
       0.14825406, 0.32390423, 0.        , 0.        , 0.        ,
       0.59690621, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.35664681, 0.        , 0.15259666, 0.30505161,
       0.        , 0.46573486, 0.        , 0.14463733, 0.        ,
       0.46669897, 0.        , 0.12281211, 0.        , 0.34152662,
       0.        , 0.16174791, 0.        , 0.18634159, 0.17352448,
       0.        , 0.        , 0.28032171, 0.        , 0.46225

You're going to define a function that takes in a VacancyID as an input and outputs a list of the 10 most similar vacancies.

In [ ]:
# Construct a reverse map of indices and VacancyIDs
indices = pd.Series(vacancies_df.index, index=vacancies_df['VacancyID']).drop_duplicates()

In [ ]:
indices[:10]

VacancyID
49442    0
31482    1
39069    2
31129    3
44258    4
29771    5
34552    6
35527    7
44603    8
46906    9
dtype: int64

####Recommending vacancies

Old

In [ ]:
jobseeker_id = 10978

# Retrieve the VacancyID of the job vacancies matched with the given JobseekerID
matches_for_jobseeker = matches_df[matches_df['JobseekerID'] == jobseeker_id]

# Sort the matches by rating in descending order
sorted_matches = matches_for_jobseeker.sort_values(by='Rating', ascending=False)

# Select the VacancyID of the top match for the jobseeker
top_match_vacancy_id = sorted_matches.iloc[0]['VacancyID']
print(f"The top match for jobseeker {jobseeker_id} is Vacancy {top_match_vacancy_id}")

# Find the row with Jobseeker and print the JobTitle and Industry columns
print(f'The characteristics of Jobseeker {jobseeker_id} are:')
row_j = jobseekers_df.loc[jobseekers_df['JobseekerID'] == jobseeker_id, ['JobseekerID', 'Industry', 'Language', 'Experience', 'Education Level']]
display(row_j)

# Find the row with the highest rated VacancyID and print the JobTitle and Industry columns
print(f'The characteristics of the highest rated Vacancy {top_match_vacancy_id} are:')
row_v = vacancies_df.loc[vacancies_df['VacancyID'] == top_match_vacancy_id, ['VacancyID','Job Title', 'Industry', 'Language', 'Experience', 'Education Level']]
display(row_v)


# Check if there are any matches for the jobseeker
if not sorted_matches.empty:
    # Get the VacancyIDs that the jobseeker has already rated
    rated_vacancy_ids = sorted_matches['VacancyID'].values

    # Define the modified get_recommendations function
    def get_recommendations(VacancyID, cosine_sim=cosine_sim, min_similarity=0.4): #ADDED: min_similarity=...
        # Get the index of the movie that matches the VacancyID
        idx = indices[VacancyID]

        # Get the pairwise similarity scores of all movies with that vacancy
        sim_scores = list(enumerate(cosine_sim[idx]))
        # ADDED: Filter the similarity scores by the minimum threshold
        sim_scores = [score for score in sim_scores if score[1] >= min_similarity]
       
        # Sort the vacancies based on the similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Get the scores and indices of the 10 most similar vacancies
        sim_scores = sim_scores[1:11]
        vacancy_indices = [i[0] for i in sim_scores] #NEW
        cosine_sim_scores = [i[1] for i in sim_scores] #NEW

        # Get the vacancy indices
        #vacancy_indices = [i[0] for i in sim_scores] REMOVED

        # Remove the VacancyIDs that the jobseeker has already rated
        vacancy_indices = [i for i in vacancy_indices if vacancies_df.iloc[i]['VacancyID'] not in rated_vacancy_ids]

        # Get the details of the most similar vacancies
        #vacancies = vacancies_df[['VacancyID', 'Job Title','Industry','Language','Experience','Education Level']].iloc[vacancy_indices]
       
        # Get the details of the most similar vacancies, including the cosine similarity scores
        vacancies = pd.DataFrame(list(zip(vacancy_indices, cosine_sim_scores)), columns=['index', 'cosine_similarity_score'])
        vacancies = vacancies.merge(vacancies_df[['VacancyID', 'Job Title','Industry','Language','Experience','Education Level']], how='left', left_on='index', right_index=True)
        
        # Return the top 10 most similar vacancies
        #return vacancies

        # Return the top 10 most similar vacancies
        return vacancies[['VacancyID', 'Job Title','Industry','Language','Experience','Education Level', 'cosine_similarity_score']]

    # Get the recommendations for the jobseeker
    recommendations = get_recommendations(top_match_vacancy_id, cosine_sim=cosine_sim, min_similarity=0.4)
  

    print(f"Recommended vacancies based on Vacancy {top_match_vacancy_id} are:")
    display(recommendations)
else:
    print(f"No matches found for jobseeker {jobseeker_id}")

The top match for jobseeker 10978 is Vacancy 44088
The characteristics of Jobseeker 10978 are:


,JobseekerID,Industry,Language,Experience,Education Level
3,10978,Construction,French,10+,High School


The characteristics of the highest rated Vacancy 44088 are:


,VacancyID,Job Title,Industry,Language,Experience,Education Level
115,44088,Finance Manager,Construction,French,10+,Master


Recommended vacancies based on Vacancy 44088 are:


,VacancyID,Job Title,Industry,Language,Experience,Education Level,cosine_similarity_score
0,34121,Finance Manager,Construction,English,10+,Master,0.883693
1,36207,Finance Manager,Construction,English,10+,Master,0.883693
2,30660,Finance Manager,Construction,German,10+,Master,0.880462
3,33680,Finance Manager,Construction,English,10+,Bachelor,0.751207
4,46107,Finance Manager,Construction,German,10+,Bachelor,0.748380
5,43627,Finance Manager,Research,French,10+,Master,0.709840
6,49003,Finance Manager,Wholesale and retail,French,10+,Master,0.703954
7,28211,Finance Manager,Agriculture,French,10+,Master,0.642154
8,31080,Finance Manager,Manufacturing,French,10+,Master,0.625831


New

In [ ]:
# Define a function to get recommendations based on the content of a given vacancy
def get_recommendations(vacancy_id, cosine_sim=cosine_sim):
    # Get the index of the vacancy that matches the vacancy_id
    idx = indices[vacancy_id]

    # Get the pairwise similarity scores between the vacancy and all other vacancies
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the vacancies based on the similarity scores in descending order
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the vacancy indices of the top 10 most similar vacancies
    sim_scores = sim_scores[1:11]
    vacancy_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar vacancies
    return list(vacancies_df['VacancyID'].iloc[vacancy_indices])

In [ ]:
# Define the feature matrix X and target variable y
X = matches_vacancies_df.drop('Rating', axis=1)
y = matches_vacancies_df['Rating']

# Create dummy variables for the categorical columns using one-hot encoding
cat_cols = ['Job Title', 'Language', 'Education Level', 'Industry','Experience', 'VacancyDescription']
X = pd.get_dummies(matches_vacancies_df[cat_cols], columns=cat_cols)

# Combine the one-hot encoded columns with the numerical columns
num_cols = ['JobseekerID', 'VacancyID', 'Rating']
X = pd.concat([X, matches_vacancies_df[num_cols]], axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Import the Logistic Regression algorithm from scikit-learn
from sklearn.linear_model import LogisticRegression

# Define the Logistic Regression algorithm
model = recommendations

# Fit the content-based recommender system to the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Compute the mean absolute error
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)

AttributeError: ignored

In [ ]:
# Make recommendations for the test data
test_users = test_df['JobseekerID'].unique()
recommendations = {}
for user_id in test_users:
    user_df = test_df[test_df['JobseekerID'] == jobseeker_id]
    recommendations[user_id] = model.recommend_items(user_df['item_id'].tolist())

# Compute the MAE for the test data
actual_ratings = []
predicted_ratings = []
for user_id in recommendations:
    user_df = test_df[test_df['user_id'] == user_id]
    actual_ratings.extend(user_df['rating'].tolist())
    predicted_ratings.extend(recommendations[user_id])
mae = mean_absolute_error(actual_ratings, predicted_ratings)
print("MAE:", mae)

NameError: ignored